## Git Clone Repo

In [ ]:
!git clone https://github.com/chenyuntc/simple-faster-rcnn-pytorch.git

Cloning into 'simple-faster-rcnn-pytorch'...
remote: Enumerating objects: 546, done.
remote: Total 546 (delta 0), reused 0 (delta 0), pack-reused 546
Receiving objects: 100% (546/546), 2.14 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (363/363), done.


## Download data

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RpzjqD048vaNzxiav0PF")
project = rf.workspace("planes-zmdv1").project("aircraft-classification-oanre")
version = project.version(5)
dataset = version.download("voc")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Aircraft-Classification-5 in voc:: 100%|██████████| 11231/11231 [01:32<00:00, 121.89it/s]


## Install necessary libraries

In [1]:
!pip install visdom scikit-image tqdm fire ipdb matplotlib torchnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu

In [2]:
%load_ext autoreload
%autoreload 2
%cd /content/drive/MyDrive/simple-faster-rcnn-pytorch
import os
import torch as t
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
%matplotlib inline

/content/drive/MyDrive/simple-faster-rcnn-pytorch


## Training

In [10]:
# train with SGD and no dropout
!python train.py train --env='fasterrcnn' --plot-every=100

# train with SGD and dropout
#!python train_Dropout.py train --env='fasterrcnn' --plot-every=100 --use-drop

# train with Adam and no dropout
# !python train_Adam.py train --env='fasterrcnn' --plot-every=100

# train with Adam and dropout
# !python train_Adam_Dropout.py train --env='fasterrcnn' --plot-every=100 --use-drop

Streaming output truncated to the last 5000 lines.
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.10/http/client.py", line 1038, in _send_output
    self.send(msg)
  File "/usr/lib/python3.10/http/client.py", line 976, in send
    self.connect()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 243, in connect
    self.sock = self._new_conn()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 218, in _new_conn
    raise NewConnectionError(
urllib3.exceptions.NewConnectionError: <urllib3.connection.HTTPConnection object at 0x7a0f45763a60>: Failed to establish a new connection: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/requests/adapters.py", line 486, in send
    resp = conn.urlopen(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpo

## Predictions

In [5]:
import os
import time
import torch
from PIL import Image
from torchvision import transforms
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
import matplotlib.pyplot as plt

# Load your trained model
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load('/content/drive/MyDrive/simple-faster-rcnn-pytorch/checkpoints/fasterrcnn_06170607.pth_0.9545454545454548')
opt.caffe_pretrain = False  # this model was trained from torchvision-pretrained model

# Function to load and preprocess the image
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.ToTensor()])
    img = transform(img).unsqueeze(0)  # Add batch dimension
    return img.cuda()

# Directory containing test images
test_images_dir = '/content/drive/MyDrive/simple-faster-rcnn-pytorch/Aircraft-Classification-2-2/test'
output_dir = '/content/drive/MyDrive/simple-faster-rcnn-pytorch/predictions'

total_time = 0
total_images = 0
# Iterate through the folder of test images
for image_name in os.listdir(test_images_dir):
    image_path = os.path.join(test_images_dir, image_name)

    if not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue  # Skip non-image files

    # Load and preprocess the image
    img = load_image(image_path)

    # Measure the inference time
    start_time = time.time()
    _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img, visualize=True)
    end_time = time.time()

    # Calculate and print the inference time
    inference_time = end_time - start_time
    print(f"Inference time for {image_name}: {inference_time} seconds")
    total_images+=1
    total_time+=inference_time

    img_array = at.tonumpy(img[0])
    bboxes = at.tonumpy(_bboxes[0])
    labels = at.tonumpy(_labels[0]).tolist()
    scores = at.tonumpy(_scores[0])

    vis_bbox(img_array, bboxes, labels, scores)

    # Save the visualized image
    plt.savefig(os.path.join(output_dir, image_name))
    plt.close()


print(f"Total inference time for {total_images} images is {total_time} seconds")
print(f"Average inference time an image is {total_time/total_images} seconds")

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 497, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "/usr/lib/python3.10/http/client.py", line 1

initialising visualiser
Exception in user code:
------------------------------------------------------------
done initialising visualiser
Inference time for a96b65_313_70_8101_2022-01-04-21-30-44_a96b65-1641331825_jpg.rf.b4d733d83f65a04a17a283b5f574ac77.jpg: 0.25415849685668945 seconds
Inference time for a98445_201_63_11615_2022-01-04-21-19-48_a98445-1641331126_jpg.rf.a41867c6249efecd43a1e1470263a75f.jpg: 0.1656181812286377 seconds
Inference time for a98445_206_67_11212_2022-01-04-21-19-43_a98445-1641331126_jpg.rf.eba5be26dc999128380d541eee51cd3f.jpg: 0.1594994068145752 seconds
Inference time for a98445_232_76_10640_2022-01-04-21-19-32_a98445-1641331126_jpg.rf.5ced7b39bff483b0d74cb7ed2798ed44.jpg: 0.16169190406799316 seconds
Inference time for a98445_232_76_10640_2022-01-04-21-19-32_a98445-1641331126_jpg.rf.ba3dec16ab3728a2afd959dcfb955cf5.jpg: 0.1765909194946289 seconds
Inference time for a98445_250_78_10559_2022-01-04-21-19-28_a98445-1641331126_jpg.rf.a4caf983d60fb2e3331d49a270171789

KeyboardInterrupt: 